In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Constant
import keras_tuner as kt
import gensim.downloader as api
import json

In [2]:
df = pd.read_csv('./ED/test.csv') # change to filepath for predictions
train_df = pd.read_csv('./ED/train.csv')

In [3]:
claims = train_df['Claim']
evidence = train_df['Evidence']

tokenizer = Tokenizer()
tokenizer.fit_on_texts(claims)
tokenizer.fit_on_texts(evidence)

claims_seq = tokenizer.texts_to_sequences(claims)
evidence_seq = tokenizer.texts_to_sequences(evidence)

CLAIMS_LEN = 13
EVIDENCE_LEN = 65
claims_seq = pad_sequences(claims_seq, maxlen=EVIDENCE_LEN, padding='post')
evidence_seq = pad_sequences(evidence_seq, maxlen=EVIDENCE_LEN, padding='post')

In [4]:
glove_vectors = api.load("glove-wiki-gigaword-100")

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = glove_vectors.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in glove_vectors:
        embedding_matrix[i] = glove_vectors[word]

In [5]:
claims = df['Claim']
evidence = df['Evidence'].fillna('')

claims_seq = tokenizer.texts_to_sequences(claims)
evidence_seq = tokenizer.texts_to_sequences(evidence)

claims_seq = pad_sequences(claims_seq, maxlen=EVIDENCE_LEN, padding='post')
evidence_seq = pad_sequences(evidence_seq, maxlen=EVIDENCE_LEN, padding='post')

In [7]:
def build_model(hp):
    claims = Input(shape=(EVIDENCE_LEN,))
    evidence = Input(shape=(EVIDENCE_LEN,))

    shared_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=EVIDENCE_LEN, trainable=False)

    x1 = shared_embedding(claims)
    x2 = shared_embedding(evidence)
    for i in range(hp.Int('num_bilstm_layers', 1, 3)):
        dropout = hp.Float(f'dropout_{i}', min_value=0, max_value=0.5, step=0.1)
        x1 = Bidirectional(LSTM(units=hp.Choice(f'lstm_units_{i}', values=[32, 64, 128, 256]),
                                dropout=dropout,
                                recurrent_dropout=dropout,
                                return_sequences=True if i < hp.get('num_bilstm_layers') - 1 else False))(x1)
        x2 = Bidirectional(LSTM(units=hp.Choice(f'lstm_units_{i}', values=[32, 64, 128, 256]),
                                dropout=dropout,
                                recurrent_dropout=dropout,
                                return_sequences=True if i < hp.get('num_bilstm_layers') - 1 else False))(x2)

    combined = Concatenate()([x1, x2])

    for k in range(hp.Int('num_dense_layers', 1, 3)):
        combined = Dense(units=hp.Choice(f'dense_units_{k}', values=[32, 64, 128, 256]),
                         activation='relu')(combined)
        combined = Dropout(rate=hp.Float(f'dense_dropout_{k}', min_value=0, max_value=0.5, step=0.1))(combined)

    output = Dense(1, activation='sigmoid')(combined)
    
    model = Model(inputs=[claims, evidence], outputs=output)
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [8]:
with open('best_hyperparameters.json', 'r') as f:
    loaded_hyperparams = json.load(f)

hyperparams = kt.HyperParameters()
for key, value in loaded_hyperparams.items():
    hyperparams.Fixed(key, value)

model = build_model(hyperparams)
model.load_weights('best.weights.h5')

In [10]:
predictions = model.predict([claims_seq, evidence_seq])

186/186 [==============================] - 11s 32ms/step - loss: 0.3696 - accuracy: 0.8250


In [ ]:
predictions = predictions.round()
predictions = predictions.astype(int)
predictions = predictions.flatten()
predictions = pd.DataFrame(predictions, columns=['prediction'])
predictions.to_csv('predictions.csv', index=False)